In [ ]:
import requests
import json
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from tqdm.auto import tqdm
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
m = Mystem()
sw = stopwords.words('russian')

In [ ]:
TOKEN = "d7a7a0c3d7a7a0c3d7a7a0c3d8d7dc6daadd7a7d7a7a0c3b59c2ba2e8acb71b6a452f29"
VERSION = "5.130"

In [ ]:
wall_get_url = "https://api.vk.com/method/wall.get"

Функция, которая скачивает записи в файл

In [ ]:
def downloader(uni_id):
    data_one = requests.get(
        wall_get_url, 
        params={
            "owner_id": -uni_id,  
            "count": 100,  
            "filter": 'owner',
            "v": VERSION, 
            "access_token": TOKEN 
        }
    ).json()
    with open(str(uni_id)+".txt", "w") as myfile:
        json.dump(data_one, myfile, ensure_ascii = False)
    data_two = requests.get(
        wall_get_url, 
        params={
            "owner_id": -uni_id, 
            "offset" : 100,
            "count": 100,  
            "filter": 'owner',
            "v": VERSION, 
            "access_token": TOKEN 
        }
    ).json()
    with open(str(uni_id)+".txt", "a") as myfile:
        json.dump(data_two, myfile, ensure_ascii = False)
    data_three = requests.get(
        wall_get_url, 
        params={
            "owner_id": -uni_id, 
            "offset" : 200,
            "count": 100,  
            "filter": 'owner',
            "v": VERSION, 
            "access_token": TOKEN 
        }
    ).json()
    with open(str(uni_id)+".txt", "a") as myfile:
        json.dump(data_three, myfile, ensure_ascii = False)

Функция, которая вырезает из документа только тексты новостей

In [ ]:
def raw_text(uni_id):
    with open(str(uni_id)+".txt", "r") as myfile:
        posts = re.findall('(?<="text": ).+?(?=",)', myfile.read())
    with open("text" + str(uni_id)+".txt", "w") as myfile:
        myfile.writelines(posts)

Функция, которая обрабатывает текст, токенизирует и приводит слова в начальную форму

In [ ]:
def parser(filename):
    with open(filename + '.txt', 'r') as myfile:
        line = myfile.read()
        words = [w.lower() for w in word_tokenize(line) if w.isalpha()]
        filtered = [w for w in words if w not in sw]
        lemmas = []
        for elem in filtered:
            if bool(re.match(r"[a-z]+", elem)) == True:
                pass
            else:
                lemma = m.lemmatize(elem)
                lemmas.append(lemma[0])
                lemmas_str = ' '.join(lemmas)
    with open('parsed_' + filename + '.txt', 'w') as myfile:
        myfile.write(lemmas_str)

Функция, которая сохранит облако слов для каждого университета

In [ ]:
def wordcloud_saver(uni):
    with open('parsed_text' + str(uni) + '.txt', 'r') as myfile:
        text = myfile.read()
        wordcloud = WordCloud(
            background_color ='white',
            width = 800,
            height = 800, 
        ).generate(text)

        plt.figure(figsize = (8, 8), facecolor = None) 
        plt.imshow(wordcloud)
        plt.axis("off") 
        plt.title('Облако слов')
        plt.savefig('/home/black/project/proga_hw/proga_project_22/static/images/cloud'+str(uni)+'.png')
        plt.close()

In [ ]:
stops = stopwords.words("russian")
tfidf = TfidfVectorizer(
    analyzer="word", 
    stop_words=stops
)

Функция, которая выделит ключевые слова, построит график их частотности и сохранит его

In [88]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

def tfidf_saver(uni):
    with open('parsed_text' + str(uni) +'.txt', 'r') as myfile:
        posts_tfidf = tfidf.fit_transform(myfile)
        feature_names = np.array(tfidf.get_feature_names_out())
        post_vector = posts_tfidf
        words = get_top_tf_idf_words(post_vector, feature_names, 10)
    with open('parsed_text' + str(uni) +'.txt', 'r') as myfile:
        data = myfile.read()
        occurences = []
        for word in words:
            count = data.count(word)
            occurences.append(count)
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(words,occurences, color='mediumpurple')
    plt.xticks(rotation = 45)
    plt.xlabel('Слова')
    plt.ylabel('Частотность')
    plt.title('Частотность ключевых слов')
    plt.savefig('/home/black/project/proga_hw/proga_project_22/static/images/freq'+str(uni)+'.jpeg')
    plt.close()
    

Выборка сообществ (топ по количеству участников):

ID – название универа

* 132 ВШЭ
* 78019879 МГУ
* 52298374 СПБГУ
* 5398 РАНХиГС
* 6319 Финансовый университет при Правительстве РФ
* 40427933 МГТУ им. Н.Э. Баумана
* 932 МФТИ
* 26555975 МГИМО


Обрабатываем все сообщества с помощью функций

In [ ]:
unis=[132,78019879,52298374,5398,6319,40427933,932,26555975]
for uni in unis:
    downloader(uni)
    raw_text(uni)
    name = 'text'+str(uni)
    parser(name)
    wordcloud_saver(uni)
    tfidf_saver(uni)

Соберём все тексты в один большой файл, чтобы построить облако слов и график для всех текстов

In [ ]:
unis=[132,78019879,52298374,5398,6319,40427933,932,26555975]
for uni in unis:
    with open('parsed_text' + str(uni) +'.txt', 'r') as myfile:
        text = myfile.read()
        with open('parsed_text01.txt', 'a') as f:
            f.write(text)

In [ ]:
uni = '01'
wordcloud_saver(uni)
tfidf_saver(uni)